In [2]:
!pip install torchvision

   ---------------------------------------- 0.0/3.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.7 MB ? eta -:--:--
   -- ------------------------------------- 0.3/3.7 MB ? eta -:--:--
   ----- ---------------------------------- 0.5/3.7 MB 1.1 MB/s eta 0:00:03
   ----- ---------------------------------- 0.5/3.7 MB 1.1 MB/s eta 0:00:03
   -------------- ------------------------- 1.3/3.7 MB 1.7 MB/s eta 0:00:02
   ------------------------- -------------- 2.4/3.7 MB 2.4 MB/s eta 0:00:01
   ------------------------------------ --- 3.4/3.7 MB 2.9 MB/s eta 0:00:01
   ---------------------------------------- 3.7/3.7 MB 2.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/111.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/111.0 MB 6.3 MB/s eta 0:00:18
    --------------------------------------- 1.8/111.0 MB 4.4 MB/s eta 0:00:25
    --------------------------------------- 2.6/111.0 MB 4.2 MB/s eta 0:00:26
   - ---------------------------

In [2]:
import torch
import torchvision
import numpy as np

print("Torch:", torch.__version__)
print("Torchvision:", torchvision.__version__)
print("CUDA available:", torch.cuda.is_available())

x = torch.randn(1, 1, 224, 224)
print("Forward test OK:", x.shape)


Torch: 2.9.1+cpu
Torchvision: 0.24.1+cpu
CUDA available: False
Forward test OK: torch.Size([1, 1, 224, 224])


In [3]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [4]:
from torchvision.datasets import ImageFolder
from torch.utils.data import Subset
from torch.utils.data import DataLoader

def reduce_dataset(dataset, fraction=0.3, seed=42):
    np.random.seed(seed)
    size = int(len(dataset) * fraction)
    indices = np.random.permutation(len(dataset))[:size]
    return Subset(dataset, indices)

train_data = ImageFolder("datasets/train", transform=transform)
test_data  = ImageFolder("datasets/test", transform=transform)

train_data = reduce_dataset(train_data, fraction=0.1)
test_data  = reduce_dataset(test_data, fraction=0.1)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_data, batch_size=32)



In [5]:
train_data

In [6]:
import torch.nn as nn
import torch.nn.functional as F

class DyslexiaCNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)

        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)

        self.fc1 = nn.Linear(128 * 28 * 28, 256)
        self.fc2 = nn.Linear(256, 3)  # 3 classes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        x = x.view(x.size(0), -1)
        x = self.dropout(F.relu(self.fc1(x)))
        return self.fc2(x)  # NO sigmoid


In [7]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DyslexiaCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)


In [15]:
!pip install tdqm

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for tdqm: filename=tdqm-0.0.1-py3-none-any.whl size=1400 sha256=5d9f20eaeac73d7f1e4990ed67dab80c941aefc1eba4d1a215ad94cbb431fca1
  Stored in directory: c:\users\oreoluwa\appdata\local\pip\cache\wheels\37\31\b8\7b711038035720ba0df14376af06e5e76b9bd61759c861ad92
Successfully built tdqm

   ---------------------------------------- 0/2 [tqdm]
   ---------------------------------------- 2/2 [tdqm]



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.47.1 requires huggingface-hub<1.0,>=0.24.0, which is not installed.
transformers 4.47.1 requires regex!=2019.12.17, which is not installed.
transformers 4.47.1 requires safetensors>=0.4.1, which is not installed.
transformers 4.47.1 requires tokenizers<0.22,>=0.21, which is not installed.


In [8]:
from tqdm import tqdm

def train_model(model, train_loader, epochs=1):
    for epoch in range(epochs):
        model.train()
        running_loss = 0

        loop = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{epochs}]")

        for images, labels in loop:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            loop.set_postfix(loss=loss.item())

        print(f"Epoch {epoch+1} Avg Loss: {running_loss/len(train_loader):.4f}")


In [9]:
train_model(model, train_loader)

Epoch [1/1]: 100%|███████████████████████████████████████████████████████| 474/474 [35:09<00:00,  4.45s/it, loss=0.295]

Epoch 1 Avg Loss: 0.5533


In [10]:
from sklearn.metrics import classification_report
import torch

def evaluate(model, test_loader):
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            outputs = model(images)

            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            y_pred.extend(preds)
            y_true.extend(labels.numpy())

    print(classification_report(
        y_true, y_pred,
        target_names=["corrected", "normal", "reversal"]
    ))


In [11]:
evaluate(model, test_loader)

              precision    recall  f1-score   support

   corrected       0.75      0.89      0.82      1961
      normal       0.74      0.72      0.73      1966
    reversal       0.85      0.71      0.78      1745

    accuracy                           0.78      5672
   macro avg       0.78      0.77      0.77      5672
weighted avg       0.78      0.78      0.77      5672



In [12]:
import torch

torch.save(model.state_dict(), "dyslexia_cnn.pth")
